<h2>Generate all_cases.csv</h2>

In [2]:
#coding:utf-8
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import os
import cn2an
import chinese2digits as c2d
import operator
import plotly.express as px
import plotly
import plotly.graph_objects as go
import plotly as py
import plotly.offline as offline
offline.init_notebook_mode(connected=True)
from urllib.request import urlopen
import json
import requests
from plotly.subplots import make_subplots
from collections import Counter
# from ipynb.fs.full.case_to_graph_only_method import convert_csv_graph, provinces_map

In [3]:
base_url = "/Users/starice/Desktop/total_extracted_result/"
pre_dir = ['type1', 'type2', 'type3', 'type4']
dir_name = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
dir_sname = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
# csv_graph, _new_csvpd = convert_csv_graph(pre_dir[:], dir_name[:], dir_sname[:])
csv_graph = nx.read_gpickle("/Users/starice/Desktop/csv_graph.gpickle")

In [61]:
plaintiff_titles = [
    '上诉人', 
    '上诉人(一审原告)', 
    '上诉人(一审第三人)', 
    '上诉人(原告)', 
    '上诉人(原审原告)', 
    '上诉人(原审原告、反诉被告)', 
    '上诉人(原审原告人)', 
    '上诉人(原审第三人)', 
    '公益诉讼起诉人', 
    '再审申请人', 
    '再审申请人(一审原告)',
    '再审申请人(一审原告、二审上诉人)',
    '再审申请人(一审原告、二审被上诉人)', 
    '再审申请人(原审原告)',
    '再审申请人(原审原告、二审上诉人)', 
    '再审申请人:(一审第三人、二审上诉人)', 
    '原告',
    '原告(反诉被告)', 
    #     '抗诉机关', 
    #     '支持起诉人', # 根据支持起诉原则，支持起诉人不能以原告的身份起诉
    #  '支持起诉机关', 
    '申诉人(一审原告、二审上诉人)',
    '申诉人(一审原告、二审上诉人、再审申请人)',
    '申诉人(一审原告、二审上诉人、原再审申请人)',
    '申诉人(一审原告、二审被上诉人)',
    '申诉人(原审原告)',
    '申请再审人(一审原告、二审上诉人)', 
    '被上诉人(一审原告)', 
    '被上诉人(原审原告)',
    '被上诉人(原审原告、反诉被告)', 
    '被上诉人(原甲原告)',
    '被上诉人一(原审原告)',
    '被申请人(一审原告、二审上诉人)',
    '被申请人(一审原告、二审被上诉人)', 
    '被申请人(原审原告)', 
    '被申请人(一审公益诉讼起诉人、二审上诉人)']

defendant_titles = [
    '(一审被告、二审被上诉人)', 
    '一审被告', 
    '一审被告(二审上诉人)', 
    '一审被告、二审被上诉人', 
    '一审被告二审上诉人)', 
    '上上诉人(原审被告)', 
    '上诉人(一审被告)', 
    '上诉人(原审第一被告)', 
    '上诉人(原审被告)',
    '上诉人(原审被告、反诉原告)',
    '上诉人(原审被告一)', 
    '上诉人(被告)', 
#     '公益诉讼出庭人', 
    '再审申请人(一审被告)',
    '再审申请人(一审被告、二审上诉人)',
    '再审申请人(一审被告、二审被上诉人)', 
    '再审申请人(再审被告)', 
    '再审申请人(原审被告)', 
    '原审当事人(原审被告)',
    '原审第三被告',
    '原审第二被告',
    '原审被告',
    '被上诉人一(原审被告一)',
    '被上诉人二(原审被告二)',
    '原审被告(反诉原告)', 
    '特别授权被告', 
    '申请再审人(原审被告)',
    '第一被告', 
    '第三被告',
    '第二被告',
    '被上诉人',
    '被上诉人(一审被告)', 
    '被上诉人(原审第三人)',
    '被上诉人(原审被告)',
    '被上诉人(原审被告、反诉原告)',
    '被上诉人(原审被告人)', 
    '被告',
    '被告(反诉原告)',
    '被告一',
    '被告二', 
    '被申诉人(一审被告,二审被上诉人)',
    '被申诉人(一审被告、二审上诉人)',
    '被申诉人(一审被告、二审被上诉人)',
    '被申诉人(一审被告、二审被上诉人、再审被申请人)',
    '被申诉人(一审被告、二审被上诉人、原再审被申请人)',
    '被申诉人(原审被告)',
    '被申请人', 
    '被申请人(一审被告)',
    '被申请人(一审被告,二审被上诉人)',
    '被申请人(一审被告、二审上诉人)',
    '被申请人(一审被告、二审被上诉人)', 
    '被申请人(原审被告)', 
    '被申请人(原审被告、二审被上诉人)'
]

cases = [(n, d) for n, d in csv_graph.nodes(data=True) \
               if d['bipartite']==0]
plaintiffs = [(e1, e2, d) for e1, e2, d in csv_graph.edges(nbunch=[n[0] for n in cases], data=True) \
                    if d['title'] in plaintiff_titles]
defendants = [(e1, e2, d) for e1, e2, d in csv_graph.edges(nbunch=[n[0] for n in cases], data=True) \
                    if d['title'] in defendant_titles]
lawyers = [(n, d) for n, d in csv_graph.nodes(data=True) \
               if (d['bipartite']==1 and d['occupation']=="律师")]
new_lawyers = [(e1, e2, d) for e1, e2, d in csv_graph.edges(nbunch=[n[0] for n in lawyers], data=True)]

# Modify lawyer name and split multiple names into multiple rows
pd_lawyers = pd.DataFrame(({"lawyer": [i[0] for i in new_lawyers], "case_id": [i[1] for i in new_lawyers]}))
pd_lawyers['lawyer_1'] = pd_lawyers['lawyer'].apply(lambda r: re.sub(r"\([^()]*\)", "", r))
pd_lawyers['lawyer_2'] = pd_lawyers['lawyer_1'].apply(lambda r: r.split("、"))
pd_lawyers = pd_lawyers.explode('lawyer_2').reset_index(drop=True)[['case_id', 'lawyer_2']]
pd_lawyers.rename(columns={"lawyer_2":"lawyer"}, inplace=True)
# print(pd_lawyers.head())

pd_plaintiffs = pd.DataFrame({"case_id": [i[0] for i in plaintiffs], "plaintiff": [i[1] for i in plaintiffs]})
pd_defendants = pd.DataFrame(({"case_id": [i[0] for i in defendants], "defendant": [i[1] for i in defendants]}))
pd_cases = pd.DataFrame({"case_id": [i[0] for i in cases], \
                        "judgement_date": [i[1]['judgement_date'] for i in cases], \
                         "is_success": [i[1]['is_success'] for i in cases], \
#                          "lat": [i[1]['lat'] for i in cases], \
#                          "lon": [i[1]['lon'] for i in cases], \
                         "court_name": [i[1]['court_name'] for i in cases], \
                         "procedure": [i[1]['procedure'] for i in cases], \
                         "judge": [i[1]['judge'] for i in cases], \
                         "legalfee": [i[1]['legalfee'] for i in cases], \
                         "objectmoney": [i[1]['objectmoney'] for i in cases], \
                         "province": [i[1]['province'] for i in cases], \
                         "city": [i[1]['city'] for i in cases], \
                         "reason": [i[1]['reason'] for i in cases], \
                         "district": [i[1]['district'] for i in cases], \
                         "penalty": [i[1]['penalty'] for i in cases]})
pd_cases['year'] = pd.DatetimeIndex(pd_cases['judgement_date']).year
pd_cases['month'] = pd.DatetimeIndex(pd_cases['judgement_date']).month
pd_cases['day'] = pd.DatetimeIndex(pd_cases['judgement_date']).day
all_cases = pd_cases.merge(pd_plaintiffs, on="case_id", how="left")
all_cases = all_cases.merge(pd_defendants, on="case_id", how="left")
all_cases = all_cases.merge(pd_lawyers, on="case_id", how="left")
all_cases.drop(all_cases[all_cases['lawyer'].str.contains('X|x|\*|之夫|之子', regex=True)==True].index, inplace=True)
all_cases.drop(all_cases[all_cases['plaintiff'].str.contains('X|x|\*|之夫|之子', regex=True)==True].index, inplace=True)
# all_cases.drop(all_cases[all_cases['plaintiff'].isin(["XX", "xx", "**"])].index, inplace=True) #delete the cases with invalid plaintiff name
all_cases.drop(all_cases[all_cases['defendant'].str.contains('X|x|\*', regex=True)==True].index, inplace=True)


In [62]:
all_cases['plaintiff'] = all_cases['plaintiff'].str.replace(r'=|;|,|。', '', regex=True)
all_cases['defendant'] = all_cases['defendant'].str.replace(r'=|;|,|。', '', regex=True)

In [63]:
all_cases.to_csv('/Users/starice/OwnFiles/cityu/RA/case_study/data/total_extracted_result/all_cases.csv', encoding="utf-8")